In [4]:
import tkinter as tk
from tkinter import messagebox
import time
import json
import os
from tkinter import simpledialog
import platform

if platform.system() == 'Windows':
    import winsound

class CountdownTimer:
    def __init__(self, master):
        self.master = master
        master.title("Countdown Timer")

        self.load_settings()
        self.running = False
        self.paused = False

        self.label = tk.Label(master, text="")
        self.label.pack()

        self.start_button = tk.Button(master, text="Start", command=self.start)
        self.start_button.pack(side=tk.LEFT)

        self.stop_button = tk.Button(master, text="Stop", command=self.stop)
        self.stop_button.pack(side=tk.LEFT)

        self.reset_button = tk.Button(master, text="Reset", command=self.reset)
        self.reset_button.pack(side=tk.LEFT)

        self.pause_button = tk.Button(master, text="Pause", command=self.pause_resume)
        self.pause_button.pack(side=tk.LEFT)

        self.set_button = tk.Button(master, text="Set Timer", command=self.set_timer)
        self.set_button.pack(side=tk.LEFT)

        self.update_label()

    def start(self):
        if not self.running:
            self.running = True
            self.update_timer()

    def stop(self):
        if self.running:
            self.running = False
            self.paused = False

    def reset(self):
        self.running = False
        self.paused = False
        self.time_left = self.default_time
        self.update_label()

    def pause_resume(self):
        if self.running:
            self.paused = not self.paused
            self.pause_button.config(text="Resume" if self.paused else "Pause")

    def set_timer(self):
        new_time = simpledialog.askinteger("Set Timer", "Enter time in seconds:", minvalue=1)
        if new_time:
            self.default_time = new_time
            self.time_left = self.default_time
            self.save_settings()
            self.update_label()

    def update_timer(self):
        if self.running and not self.paused:
            self.update_label()
            if self.time_left > 0:
                self.time_left -= 1
                self.master.after(1000, self.update_timer)
            else:
                self.play_sound()
                messagebox.showinfo("Time's Up!", "Countdown finished!")
                self.reset()

    def update_label(self):
        minutes, seconds = divmod(self.time_left, 60)
        self.label.config(text=f"{minutes:02d}:{seconds:02d}")

    def load_settings(self):
        if os.path.exists('settings.json'):
            with open('settings.json', 'r') as f:
                settings = json.load(f)
                self.default_time = settings.get('default_time', 60)
        else:
            self.default_time = 60
        self.time_left = self.default_time

    def save_settings(self):
        settings = {
            'default_time': self.default_time
        }
        with open('settings.json', 'w') as f:
            json.dump(settings, f)

    def play_sound(self):
        if platform.system() == 'Windows':
            winsound.PlaySound('SystemExit', winsound.SND_ALIAS)
        elif platform.system() == 'Darwin':  # macOS
            os.system('afplay /System/Library/Sounds/Glass.aiff')
        else:
            print("Time's up!")  

if __name__ == "__main__":
    root = tk.Tk()
    timer = CountdownTimer(root)
    root.mainloop()
